In [1]:
!pip install pandas scikit-learn
!pip install anthropic

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MISTRAL_KEY= os.getenv("MISTRAL_KEY")

In [3]:
import pathlib

BASE_DIR = pathlib.Path.cwd().resolve().parents[1]
DATA_DIR = BASE_DIR /"data"
OUT_DIR = BASE_DIR / "inference" / "outputs"

DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)
print(BASE_DIR)

/Users/raimanawal/Downloads/nlp fin/FINAL/llm-eval


## Slicing dataset aNd splitting into training and validation sets

In [4]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("TIGER-Lab/MMLU-Pro", split="test[:1000]")
df = dataset.to_pandas()

df = df[["question_id", "question", "options", "answer"]].rename(columns={"question_id": "id", "options": "choices"})
df.to_csv(DATA_DIR / "mmlu_pro_1k.csv", index=False)

print("Cleaned and saved:", len(df), "rows →", DATA_DIR / "mmlu_pro_1k.csv")
df.head()

/Users/raimanawal/miniforge3/envs/llm-eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cleaned and saved: 1000 rows → /Users/raimanawal/Downloads/nlp fin/FINAL/llm-eval/data/mmlu_pro_1k.csv


,id,question,choices,answer
0,70,"Typical advertising regulatory bodies suggest,...","[Safe practices, Fear, Jealousy, Trivial, Unsa...",I
1,71,Managers are entrusted to run the company in t...,"[Shareholders, Diligence, Self-interest, Share...",F
2,72,There are two main issues associated with ____...,"[Down, Autonomy, Remuneration, Benefit, Down, ...",J
3,73,_______ locate morality beyond the sphere of r...,"[Ethical egoism, Ethics of duty, Postmodern et...",C
4,74,Some of key differences between Islamic finan...,"[Interest, Certain, Assured, Both tangible and...",G


In [5]:
from sklearn.model_selection import train_test_split

calib_df, eval_df = train_test_split(df, test_size=0.85, random_state=42, shuffle=True)
calib_df.to_csv(DATA_DIR / "calibration_split.csv", index=False)
eval_df.to_csv(DATA_DIR / "evaluation_split.csv", index=False)

print("Split complete:")
print("Calibration:", len(calib_df), "rows")
print("Evaluation :", len(eval_df), "rows")

Split complete:
Calibration: 150 rows
Evaluation : 850 rows


In [6]:
print("Calibration sample:")
display(calib_df.head(3))

print("Evaluation sample:")
display(eval_df.head(3))

Calibration sample:


,id,question,choices,answer
32,102,Find the amount to be paid each month in order...,"[$275.50, $6826.25, $8371.42, $344.42, $310.22...",J
779,856,Four high school and college friends started a...,"[12, 15, 11, 7, 16, 13, 8, 9, 10, 14]",I
839,916,A shop owner domiciled in State A sued a distr...,"[No, because the distributor and the wholesale...",H


Evaluation sample:


,id,question,choices,answer
521,596,A manufacturer is currently selling 2000 units...,"[$2.50, $1.90, $2.70, $2.60, $1.80, $2.20, $2....",F
737,814,Fred Lowes is a typewriter salesman. He receiv...,"[$210, $200, $225, $175, $195, $150, $255, $24...",I
740,817,"Mary Redmond purchased a $28,500 home with 20%...","[$305, $190, $171, $285.50, $399, $323, $5700,...",F


In [7]:
import numpy as np
import pandas as pd

sim_df = eval_df.copy()
rng = np.random.default_rng(42)
all_choices = [list(c) for c in sim_df["choices"]]
sim_df["pred"] = [rng.choice(choices) for choices in all_choices]
sim_df["conf"] = rng.uniform(0.4, 1.0, size=len(sim_df)).round(3)

MODEL_NAME = "simulated_model"   
DATASET_FILENAME = "mmlu_pro_1k.csv"

OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_FILENAME.split('.')[0]}_by_threshold.csv"
sim_df.to_csv(OUTPUT_PATH,index=False)

print("Simulated predictions created →", OUT_DIR / "simulated_model_outputs.csv")
sim_df.head()

Simulated predictions created → /Users/raimanawal/Downloads/nlp fin/FINAL/llm-eval/inference/outputs/simulated_model_outputs.csv


,id,question,choices,answer,pred,conf
521,596,A manufacturer is currently selling 2000 units...,"[$2.50, $1.90, $2.70, $2.60, $1.80, $2.20, $2....",F,$2.50,0.881
737,814,Fred Lowes is a typewriter salesman. He receiv...,"[$210, $200, $225, $175, $195, $150, $255, $24...",I,$240,0.543
740,817,"Mary Redmond purchased a $28,500 home with 20%...","[$305, $190, $171, $285.50, $399, $323, $5700,...",F,$5700,0.910
660,737,Paul owed $56.30 plus a 3% sales tax to a spor...,"[$57.99, $60.35, $58.65, $59.50, $57.50, $56.8...",F,$57.50,0.434
411,485,An automobile dealer wishes to sell one of his...,"[$1339.85, $76.50, $70, $77.77, $74.44, $65.00...",E,$74.44,0.881


## QWEN

### Please run n check this 


In [ ]:
# !pip install -q -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install -q transformers accelerate bitsandbytes pandas
# # !pip install -q --upgrade transformers accelerate bitsandbytes pandas==2.2.2
# # Verify installation
# import torch
# print(f"PyTorch version: {torch.__version__}")
# print(f"CUDA available: {torch.cuda.is_available()}")
# print(f"CUDA version: {torch.version.cuda}")

In [ ]:

# import torch
# import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import gc

# major, minor = map(int, transformers.__version__.split('.')[:2])
# if (major, minor) < (4, 37):
#     raise ValueError(f"Transformers {transformers.__version__} too old. Need 4.37+.")

# MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
# print(f"Loading: {MODEL_NAME}")

# if torch.backends.mps.is_available():
#     device = torch.device("mps")
#     dtype = torch.float16
# else:
#     device = torch.device("cpu")
#     dtype = torch.float32

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=dtype,
#     device_map=None, 
#     token=HF_TOKEN
# ).to(device)

# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

# print(f"Model loaded successfully on {device} with dtype={dtype}")


In [ ]:

# import pandas as pd
# import re
# import time
# from collections import Counter
# import torch
# import gc

# def extract_letter(text):
#     text_clean = text.strip().upper()

#     if "I DON'T KNOW" in text_clean:
#         return "IDK"

#     m = re.search(r"\b([A-D])\b", text_clean)
#     return m.group(1) if m else None


# def qwen_answer_and_conf(prompt, n=6, temperature=0.7):
#     votes = []
#     for _ in range(n):
#         try:
#             messages = [{"role": "user", "content": prompt}]
#             formatted_prompt = tokenizer.apply_chat_template(
#                 messages, tokenize=False, add_generation_prompt=True
#             )

#             inputs = tokenizer(
#                 formatted_prompt,
#                 return_tensors="pt",
#                 truncation=True,
#                 max_length=512
#             ).to(model.device)

#             with torch.no_grad():
#                 outputs = model.generate(
#                     **inputs,
#                     max_new_tokens=10,
#                     temperature=temperature,
#                     do_sample=True if temperature > 0 else False,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id
#                 )

#             txt = tokenizer.decode(
#                 outputs[0][inputs['input_ids'].shape[1]:],
#                 skip_special_tokens=True
#             ).strip()

#             letter = extract_letter(txt)
#             if letter:
#                 votes.append(letter)

#             del inputs, outputs
#             torch.cuda.empty_cache()

#         except Exception as e:
#             print("Error:", e)

#         time.sleep(0.1)

#     if not votes:
#         return "IDK", 0.0

#     counts = Counter(votes)
#     pred = counts.most_common(1)[0][0]
#     conf = counts[pred] / len(votes)
#     return pred, float(conf)


# subset_df = pd.read_csv("outputs/week7/mmlu_full_all_thresholds.csv")
# print(f"Loaded {len(subset_df)} examples from all thresholds")

# qwen_rows = []
# total_q = len(subset_df)

# for i, row in subset_df.iterrows():
#     prompt = row["prompt"]
#     t = row["threshold"]

#     print(f"[t={t} | {i+1}/{total_q}] Asking Qwen2.5-1.5B ...")
#     pred, conf = qwen_answer_and_conf(prompt, n=6)

#     qwen_rows.append({
#         "id": row["id"],
#         "threshold": t,  # <--- include threshold
#         "question": row["question"],
#         "choices": row["choices"],
#         "answer": row["answer"],
#         "predicted_answer": pred,
#         "confidence": conf
#     })

#     if (i + 1) % 5 == 0:
#         gc.collect()
#         torch.cuda.empty_cache()

# qwen_out = pd.DataFrame(qwen_rows)
# qwen_path= OUT_DIR / f"qwen-mmlu.csv"
# qwen_out.to_csv(qwen_path, index=False)
# print(f"Qwen2.5-1.5B predictions for all thresholds saved → {qwen_path}")
# qwen_out.head(5)

## GPT-4o-mini

In [ ]:
from openai import OpenAI
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter

client = OpenAI(api_key=OPENAI_API_KEY)

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

MODEL_NAME = "gpt-4o-mini"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"gpt-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv →", len(eval_df), "rows")

def build_mcq_prompt(row, t):
    opts = row["choices"]
    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except Exception:
            pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]
    if isinstance(opts, (list, tuple)):
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))

    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )

    return "\n".join(lines)

def extract_letter(text):
    clean = text.strip().upper()
    if "I DON'T KNOW" in clean:
        return "IDK"
    match = re.search(r"\b([A-Z])\b", clean)
    return match.group(1) if match else None


def gpt_answer_and_conf(prompt, n=6, temperature=0.7, sleep_s=0.4):
    votes = []
    for _ in range(n):
        try:
            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=10
            )
            txt = resp.choices[0].message.content.strip()
            letter = extract_letter(txt)
            if letter:
                votes.append(letter)
        except Exception as e:
            print("GPT Error:", e)

        time.sleep(sleep_s)

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)


if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)

for t in THRESHOLDS:
    print(f"\n=== Threshold t={t} ===")

    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue

        prompt = build_mcq_prompt(row, t)
        print(f"[t={t}] {idx+1}/{total} → GPT...")

        pred, conf = gpt_answer_and_conf(prompt)

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })
        done_pairs.add(key)
        new_since_save += 1

        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"💾 Saved {len(saved_rows)} rows.")
            new_since_save = 0


pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("saved:", OUTPUT_PATH)

In [ ]:
print("done")

In [ ]:
import os
import shutil

downloads_path = os.path.expanduser("~/Downloads")
local_copy_path = os.path.join(downloads_path, os.path.basename(OUTPUT_PATH))

shutil.copy(OUTPUT_PATH, local_copy_path)

print("copied to Downloads:", local_copy_path)


## Gemini

In [ ]:

import google.generativeai as genai
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

MODEL_NAME = "gemini-"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv →", len(eval_df), "rows")

def build_mcq_prompt(row, t):
    opts = row["choices"]

    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except:
            pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]

    if isinstance(opts, (list, tuple)):
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))

    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )

    return "\n".join(lines)


def extract_gemini_text(resp):
    try:
        return resp.text
    except:
        pass

    try:
        return resp.candidates[0].content.parts[0].text
    except:
        return ""

def extract_letter(text):
    clean = text.strip().upper()
    if "I DON'T KNOW" in clean:
        return "IDK"
    m = re.search(r"\b([A-Z])\b", clean)
    return m.group(1) if m else None


gemini_model = genai.GenerativeModel(MODEL_NAME)


def gemini_answer_and_conf(prompt, n=6, temperature=0.7, base_sleep=1.5):
    """
    A safe version with:
    - auto-retry on 429 rate limits
    - sleeps between calls
    - returns predicted letter + confidence
    """
    votes = []
    for _ in range(n):
        while True:
            try:
                resp = gemini_model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=temperature,
                        max_output_tokens=10
                    )
                )
                txt = extract_gemini_text(resp).strip()
                letter = extract_letter(txt)
                if letter:
                    votes.append(letter)
                time.sleep(base_sleep)
                break
            except Exception as e:
                err = str(e)
                print("Gemini Error:", err)

                if "429" in err or "Quota exceeded" in err:
                    retry_sec = 45
                    m = re.search(r"retry_delay.*?(\d+)", err)
                    if m:
                        retry_sec = int(m.group(1))
                    print(f"Rate limit hit. Sleeping {retry_sec} seconds…")
                    time.sleep(retry_sec)
                    continue
                print("Non-quota error. Sleeping 5 seconds…")
                time.sleep(5)
                continue
    if not votes:
        return "IDK", 0.0
    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)

if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)


for t in THRESHOLDS:
    print(f"\n=== Threshold t={t} ===")
    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue
        prompt = build_mcq_prompt(row, t)
        print(f"[t={t}] {idx+1}/{total} → Gemini...")
        pred, conf = gemini_answer_and_conf(prompt)
        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })
        done_pairs.add(key)
        new_since_save += 1
        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"💾 Saved {len(saved_rows)} rows.")
            new_since_save = 0
pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("saved:", OUTPUT_PATH)


In [ ]:
import pandas as pd
import numpy as np

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

MODEL_NAME = "gemini"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"
print("Checking file:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
res_df = pd.read_csv(OUTPUT_PATH)

print("eval_df rows:", len(eval_df))
print("result rows:", len(res_df))

res_df["threshold"] = res_df["threshold"].astype(float)
dups = (
    res_df.groupby(["id", "threshold"])
          .size()
          .reset_index(name="count")
          .query("count > 1")
)
print("Duplicate (id, threshold) rows:", len(dups))

eval_ids = set(eval_df["id"])
res_ids = set(res_df["id"])

missing_ids = eval_ids - res_ids
extra_ids = res_ids - eval_ids

print("Missing IDs:", len(missing_ids))
print("Extra IDs:", len(extra_ids))

missing_pairs = []
extra_pairs = []

for qid in eval_ids:
    present = set(res_df.loc[res_df["id"] == qid, "threshold"])
    for t in THRESHOLDS:
        if t not in present:
            missing_pairs.append((qid, t))
    for t in present:
        if t not in THRESHOLDS:
            extra_pairs.append((qid, t))

print("Missing (id, threshold) combos:", len(missing_pairs))
print("Extra (id, threshold) combos:", len(extra_pairs))

valid_letters = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
valid_answers = valid_letters | {"IDK"}

invalid_pred = res_df[~res_df["predicted_answer"].astype(str).isin(valid_answers)]
print("Invalid predicted_answer rows:", len(invalid_pred))

invalid_conf = res_df[
    (res_df["confidence"].isna()) |
    (res_df["confidence"] < 0) |
    (res_df["confidence"] > 1)
]
print("Invalid confidence rows:", len(invalid_conf))

if (
    len(dups) == 0 and
    len(missing_ids) == 0 and
    len(extra_ids) == 0 and
    len(missing_pairs) == 0 and
    len(extra_pairs) == 0 and
    len(invalid_pred) == 0 and
    len(invalid_conf) == 0
):
    print("\nAll sanity checks passed!")
else:
    print("\nIssues detected — see counts above.")


## Mistral

In [11]:
import os
import mistralai
import re

# 1. Load API key
MISTRAL_KEY = os.getenv("MISTRAL_KEY")
if not MISTRAL_KEY:
    raise ValueError("❌ MISTRAL_KEY not found — add it to your .env and restart kernel.")

# 2. Create client
client = mistralai.Mistral(api_key=MISTRAL_KEY)

# 3. Model
MODEL_NAME = "open-mistral-7b"
print("🚀 Testing Mistral API connection…")
print(f"Using model: {MODEL_NAME}")

# 4. A simple test prompt
prompt = """Q: What color is the sky on a clear day?
Options:
A. Red
B. Blue
C. Green
D. Yellow

Answer ONLY the letter or 'I don't know'."""

# 5. Request
try:
    resp = client.chat.complete(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=10,
    )
except Exception as e:
    print("\n❌ API Error:", str(e))
    raise

# 6. Extract text using ATTRIBUTE ACCESS
raw_text = resp.choices[0].message.content
print("\nRaw Response:")
print(raw_text)

# 7. Extract letter
clean = raw_text.strip().upper()

if "I DON'T KNOW" in clean or "IDK" in clean:
    pred = "IDK"
else:
    m = re.search(r"\b([A-Z])\b", clean)
    pred = m.group(1) if m else None

print("\nExtracted Prediction:", pred)

🚀 Testing Mistral API connection…
Using model: open-mistral-7b

Raw Response:
B. Blue

Extracted Prediction: B


In [13]:
import mistralai
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter

# MISTRAL_KEY = os.getenv("MISTRAL_KEY")
# if not MISTRAL_KEY:
#     raise ValueError("Missing MISTRAL_KEY in environment (.env file).")

client = mistralai.Mistral(api_key=MISTRAL_KEY)

MODEL_NAME = "open-mistral-7b"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv →", len(eval_df), "rows")

def build_mcq_prompt(row, t):
    opts = row["choices"]

    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except:
            pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]

    if isinstance(opts, (list, tuple)):
        for i, c in enumerate(opts):
            lines.append(f"{labels[i]}. {c}")
    else:
        lines.append(str(opts))

    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points. "
        "Correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )
    return "\n".join(lines)


def extract_letter(text):
    clean = text.strip().upper()

    if "I DON'T KNOW" in clean or clean == "IDK" or "IDK" in clean:
        return "IDK"

    m = re.search(r"\b([A-Z])\b", clean)
    return m.group(1) if m else None

def mistral_answer_and_conf(prompt, n=6, temperature=0.7, base_sleep=1.2):
    """
    n = number of samples
    base_sleep = cooldown between API calls
    """
    votes = []

    for _ in range(n):
        while True:
            try:
                resp = client.chat.complete(
                    model=MODEL_NAME,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=temperature,
                    max_tokens=10,
                )

                txt = resp.choices[0].message.content.strip()
                letter = extract_letter(txt)

                if letter:
                    votes.append(letter)

                time.sleep(base_sleep)
                break

            except Exception as e:
                err = str(e)
                print("Mistral Error:", err)

                if "429" in err or "rate" in err.lower():
                    print("Rate limit hit — sleeping 30 sec.")
                    time.sleep(30)
                    continue

                print("Unexpected error — sleeping 5 sec.")
                time.sleep(5)
                continue

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, conf

if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

for t in THRESHOLDS:
    print(f"\n=== Threshold t={t} ===")

    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue

        prompt = build_mcq_prompt(row, t)
        print(f"[t={t}] {idx+1}/{total} → Mistral API...")

        pred, conf = mistral_answer_and_conf(prompt)

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })

        done_pairs.add(key)
        new_since_save += 1

        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"💾 Saved {len(saved_rows)} rows.")
            new_since_save = 0

pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("saved:", OUTPUT_PATH)


Saving to: /Users/raimanawal/Downloads/nlp fin/FINAL/llm-eval/inference/outputs/open-mistral-7b-mmlu.csv
Loaded evaluation_split.csv → 850 rows
Starting fresh.

=== Threshold t=0.25 ===
[t=0.25] 1/850 → Mistral API...
[t=0.25] 2/850 → Mistral API...
[t=0.25] 3/850 → Mistral API...
[t=0.25] 4/850 → Mistral API...
[t=0.25] 5/850 → Mistral API...
[t=0.25] 6/850 → Mistral API...
[t=0.25] 7/850 → Mistral API...
[t=0.25] 8/850 → Mistral API...
[t=0.25] 9/850 → Mistral API...
[t=0.25] 10/850 → Mistral API...
💾 Saved 10 rows.
[t=0.25] 11/850 → Mistral API...
[t=0.25] 12/850 → Mistral API...
[t=0.25] 13/850 → Mistral API...
[t=0.25] 14/850 → Mistral API...
[t=0.25] 15/850 → Mistral API...
[t=0.25] 16/850 → Mistral API...
[t=0.25] 17/850 → Mistral API...
[t=0.25] 18/850 → Mistral API...
[t=0.25] 19/850 → Mistral API...
[t=0.25] 20/850 → Mistral API...
💾 Saved 20 rows.
[t=0.25] 21/850 → Mistral API...
[t=0.25] 22/850 → Mistral API...
[t=0.25] 23/850 → Mistral API...
[t=0.25] 24/850 → Mistral API.

## llama

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, os

MODEL_NAME = "meta-llama/Llama-3.2-3B"
DTYPE = torch.bfloat16
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"

print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=os.getenv("HF_TOKEN"),
    trust_remote_code=True
)

print("\nLoading model (bf16 on MPS, device_map='auto')...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=os.getenv("HF_TOKEN"),
    torch_dtype=DTYPE,
    device_map="auto",
    trust_remote_code=True
)

model.eval()
print("Loaded ✓")

In [ ]:
import os
import ast
import re
import torch
import torch.nn.functional as F
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN is None:
    raise ValueError("HF_TOKEN missing in .env")

BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR.parent.parent / "data"
OUT_DIR = BASE_DIR.parent.parent / "outputs"
OUT_DIR.mkdir(exist_ok=True)


DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
DTYPE = torch.bfloat16

print("Device:", DEVICE, "| dtype:", DTYPE)

MODEL_NAME = "meta-llama/Llama-3.2-3B"

print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    trust_remote_code=True
)

print("Loading model (bf16 on MPS, device_map='auto')...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    torch_dtype=DTYPE,
    device_map="auto",
    trust_remote_code=True
)

model.eval()
print("Model loaded.")

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded:", len(eval_df), "rows")

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]
SAVE_EVERY = 10

OUTPUT_PATH = OUT_DIR / f"llama-mmlu.csv"

def build_mcq_prompt(row, t):
    opts = row["choices"]
    if isinstance(opts, str):
        try: opts = ast.literal_eval(opts)
        except: pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    text = [f"Q: {row['question']}", "Options:"]
    for i, ch in enumerate(opts):
        text.append(f"{labels[i]}. {ch}")

    text.append(
        f"\nOnly answer if >{t} confident. Wrong = penalty {t}/(1−{t}), correct=+1, IDK=0.\n"
        "If unsure respond EXACTLY: 'I don't know'.\n"
        "Return ONLY a letter (A, B, C, …) OR 'I don't know'."
    )
    return "\n".join(text)


def extract_letter(text):
    text = text.strip().upper()
    if "I DON'T KNOW" in text:
        return "IDK"
    m = re.search(r"\b([A-Z])\b", text)
    return m.group(1) if m else None


def model_answer_and_conf(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[:, -1, :]   # only final token

    probs = torch.softmax(logits, dim=-1)

    pred_id = torch.argmax(logits, dim=-1).item()
    pred_text = tokenizer.decode(pred_id)
    letter = extract_letter(pred_text)

    if letter and letter != "IDK":
        tid = tokenizer(letter, add_special_tokens=False).input_ids[0]
        confidence = probs[0, tid].item()
    else:
        confidence = 0.0

    return letter if letter else "IDK", confidence


if OUTPUT_PATH.exists():
    old = pd.read_csv(OUTPUT_PATH)
    saved_rows = old.to_dict("records")
    done = set(zip(old["id"], old["threshold"]))
    print("Resuming from:", len(old))
else:
    saved_rows = []
    done = set()
    print("Starting fresh")

save_counter = 0

for t in THRESHOLDS:
    print(f"\n=== t = {t} ===")

    for idx, row in eval_df.iterrows():

        key = (row["id"], t)
        if key in done:
            continue

        prompt = build_mcq_prompt(row, t)
        pred, conf = model_answer_and_conf(prompt)

        final_pred = pred if conf >= t else "IDK"

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"], 
            "choices": row["choices"],
            "answer": row["answer"],
            "raw_prediction": pred,
            "confidence": conf,
            "thresholded_prediction": final_pred
        })

        done.add(key)
        save_counter += 1

        if save_counter >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print("Saved", len(saved_rows))
            save_counter = 0

pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("\nDONE →", OUTPUT_PATH)

## Claude

In [ ]:
from anthropic import Anthropic
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

MODEL_NAME = "claude"
DATASET_NAME = "mmlu"

# Output path
OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

# Load evaluation split
eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv →", len(eval_df), "rows")


Saving to: /Users/stutisinghal/Documents/fall25/NLP/final/final_branch/cloned_final/llm-eval/inference/outputs/claude-mmlu.csv
Loaded evaluation_split.csv → 850 rows


In [99]:
def build_mcq_prompt(row, t):
    opts = row["choices"]

    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except:
            pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]

    if isinstance(opts, (list, tuple)):
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))

    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )

    return "\n".join(lines)


In [100]:
def extract_claude_text(resp):
    try:
        return resp.content[0].text
    except:
        return ""


In [101]:
def extract_letter(text):
    clean = text.strip().upper()
    if "I DON'T KNOW" in clean:
        return "IDK"
    m = re.search(r"\b([A-Z])\b", clean)
    return m.group(1) if m else None


In [102]:
def claude_answer_and_conf(prompt, n=6, temperature=0.7, base_sleep=1.2):
    votes = []

    for _ in range(n):
        while True:
            try:
                resp = client.messages.create(
                    model="claude-haiku-4-5",
                    max_tokens=20,
                    temperature=temperature,
                    messages=[
                        {"role": "user", "content": prompt}
                    ]
                )

                txt = extract_claude_text(resp).strip()
                letter = extract_letter(txt)
                if letter:
                    votes.append(letter)

                time.sleep(base_sleep)
                break

            except Exception as e:
                err = str(e)
                print("Claude Error:", err)

                # Rate limits
                if "rate_limit" in err or "429" in err:
                    print("🔁 Rate limit — sleeping 30s…")
                    time.sleep(30)
                    continue

                # Generic temporary errors
                print("⚠️ Non-rate error — sleeping 5s…")
                time.sleep(5)
                continue

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)


In [103]:
if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)


Resuming — loaded 2310 rows.


In [104]:
for t in THRESHOLDS:
    print(f"\n=== Threshold t={t} ===")

    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue

        prompt = build_mcq_prompt(row, t)

        print(f"[t={t}] {idx+1}/{total} → Claude...")

        pred, conf = claude_answer_and_conf(prompt)

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })

        done_pairs.add(key)
        new_since_save += 1

        # SAVE EVERY 10
        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"💾 Saved {len(saved_rows)} rows.")
            new_since_save = 0

# Final save
pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("saved:", OUTPUT_PATH)



=== Threshold t=0.25 ===

=== Threshold t=0.5 ===

=== Threshold t=0.75 ===
[t=0.75] 611/850 → Claude...
[t=0.75] 612/850 → Claude...
[t=0.75] 613/850 → Claude...
[t=0.75] 614/850 → Claude...
[t=0.75] 615/850 → Claude...
[t=0.75] 616/850 → Claude...
[t=0.75] 617/850 → Claude...
[t=0.75] 618/850 → Claude...
[t=0.75] 619/850 → Claude...
[t=0.75] 620/850 → Claude...
💾 Saved 2320 rows.
[t=0.75] 621/850 → Claude...
[t=0.75] 622/850 → Claude...
[t=0.75] 623/850 → Claude...
[t=0.75] 624/850 → Claude...
[t=0.75] 625/850 → Claude...
[t=0.75] 626/850 → Claude...
[t=0.75] 627/850 → Claude...
[t=0.75] 628/850 → Claude...
[t=0.75] 629/850 → Claude...
[t=0.75] 630/850 → Claude...
💾 Saved 2330 rows.
[t=0.75] 631/850 → Claude...
[t=0.75] 632/850 → Claude...
[t=0.75] 633/850 → Claude...
[t=0.75] 634/850 → Claude...
[t=0.75] 635/850 → Claude...
[t=0.75] 636/850 → Claude...
[t=0.75] 637/850 → Claude...
[t=0.75] 638/850 → Claude...
[t=0.75] 639/850 → Claude...
[t=0.75] 640/850 → Claude...
💾 Saved 2340 ro